In [2]:
#importing neccesary packages
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

# Importing Training and Test Data

In [49]:
#Reading movielens data from 

columns_names = ["user_id","movie_id","rating","timestamp"]
train = pd.read_csv("ml-100k/u5.base",sep = '\t',names = columns_names)
test = pd.read_csv("ml-100k/u5.test",sep = '\t',names = columns_names)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   user_id    80000 non-null  int64
 1   movie_id   80000 non-null  int64
 2   rating     80000 non-null  int64
 3   timestamp  80000 non-null  int64
dtypes: int64(4)
memory usage: 2.4 MB


# Pivot Table

In [55]:
movie_matrix=np.zeros((1682,943))
total = len(train)
for i in range (total):
    u_id = train["user_id"][i]-1
    m_id = train["movie_id"][i]-1
    rating= train["rating"][i]
    movie_matrix[m_id][u_id]= rating
    

In [39]:
movie_matrix.shape


(1682, 943)

# Similarity Matrix

In [56]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(movie_matrix)
similarity_matrix.shape

(1682, 1682)

# Computing Ratings for the given User -Training and Testing

In [72]:

K=[10,20,30,40]
MAE=[]
for k in K:
    mae=0
    num_users=0
    for i in tqdm(range(len(test))):
        
        u_id = test["user_id"][i]-1
        m_id = test["movie_id"][i]-1
        actual_rating= test["rating"][i]
        
        predicted_rating=0
        total_sim=0
        similarity=[]
        rating=[]
        movie=[]
        
        for j in range(1682):
            if(j==m_id):
                continue
            if(movie_matrix[j][u_id]==0):
                continue
            similarity.append(similarity_matrix[j][m_id])
            rating.append(movie_matrix[j][u_id]) 
            
        top_k=sorted(range(len(similarity)), key=lambda i: similarity[i])[-k:]
        
        for p in top_k:
            predicted_rating+=rating[p]*similarity[p]
            total_sim += similarity[p]
            
        if total_sim != 0:
            #print(rating)
            mae += abs(actual_rating-(predicted_rating/total_sim))
            num_users+=1
        else:
            continue
            
    mae/=num_users
    MAE.append(mae)  

100%|██████████| 20000/20000 [00:27<00:00, 726.21it/s]


In [58]:
MAE

[0.7612224165709063,
 0.7643593168820328,
 0.7715358298339332,
 0.7777421083064774]

In [74]:
MAE

[0.7612224165709063,
 0.7643593168820328,
 0.7715358298339332,
 0.7777421083064774]